In [1]:
from Keys import my_lang_api_key, my_google_api_key
import os
os.environ["GOOGLE_API_KEY"] = my_google_api_key
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_PROJECT']="PortfoliAI_v1"
os.environ['LANGCHAIN_API_KEY'] = my_lang_api_key

import langchain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain_core.runnables.base import RunnableEach
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from IPython.display import Markdown


In [8]:
from Keys import my_lang_api_key, my_google_api_key
import os
os.environ["GOOGLE_API_KEY"] = my_google_api_key
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGSMITH_PROJECT']="PortfoliAI_v1"
os.environ['LANGCHAIN_API_KEY'] = my_lang_api_key

In [70]:
from markdown import markdown
from IPython.display import Markdown

In [54]:
import langchain_core.runnables.base as runnable

In [2]:
from langchain_core.runnables.base import RunnableEach

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
import langchain

In [2]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.5,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [12]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.vectorstores import FAISS

In [5]:
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [6]:
vector_db=FAISS.load_local("faiss_vector_db",embeddings,allow_dangerous_deserialization=True)

## RetrievalQAWithSourcesChain

In [15]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vector_db.as_retriever())

In [16]:
my_query="Describe his projects that involve python."

In [17]:
vector_db.similarity_search(query=my_query,k=5)

[Document(metadata={'source': 'C://Users//ASUS//Documents//Python Scripts//LnE Langchain//RAG Documents//Ankon Bhowmick Resume.docx'}, page_content='Estimating Public Opinion on Gun Legislation using MRP:\n\nEstimated public opinion on a ban on assault rifles in the US at the state level using a singular national survey.\n\nCollected survey data and population data and conducted data cleaning, transformation, and data matching to created input dataset and poststratification frame respectively using the Python packages Pandas and NumPy.\n\nModelled hierarchical relationships between individual-level and group-level predictors using rstanarm package in R and analysed interactions and state-level predictors using posterior distributions.\n\nValidated model performance using split sample validation and disaggregation for varying MRP sample sizes. Evaluated model performance using MAE, Correlation, and WAIC.\n\nPoststratified for final state-wise predictions of percentage opposition to a ba

In [19]:
answer=chain.invoke({"question": my_query})

In [22]:
print(answer['answer'])

His projects involving Python include:

*   **Data Analysis and Presentation**: He collected survey and population data, performing data cleaning, transformation, and matching using Python packages Pandas and NumPy. He presented the results using matplotlib in Python.
*   **Quantitative Momentum Investing Model**: He established a data pipeline for cleaning, transforming, and integrating data from individual company files using pandas and NumPy.
*   **Image Classification and Captioning**: He constructed basic Neural Network (NN) and Convolutional Neural Network (CNN) classification models using TensorFlow and PyTorch. He also built an encoder-decoder architecture-based model to generate captions for images, using an encoder network to convert images into feature vectors and a decoder network to create captions.
*   **Predicting the Course of the Russo-Ukrainian War**: He collected data through web scraping using Selenium and Beautiful Soup packages in Python. He preprocessed articles 

## Prompts and Retriever

In [7]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

In [8]:
system_prompt = SystemMessagePromptTemplate.from_template(
    "You are an AI assistant that answers queries about a candidate."
)

In [9]:
prompt_rag_human = HumanMessagePromptTemplate.from_template(
    """You are tasked with drafting an answer for a question asked regarding a candidate's skills and experience.

The answer should only contain information on the candidate in the form of context that has been made available to you, do not falsify any skills or experience. 
For behavourial questions, use slightly creative vocabulary to best answer them.
The context is: {context}
The question is: {question}""",
    input_variables=["question"]
)

In [10]:
prompt_rag = ChatPromptTemplate.from_messages([system_prompt, prompt_rag_human])

In [11]:
retriever=vector_db.as_retriever(search_kwargs={"k": 3})

In [ ]:
docs_ret=vector_db.similarity_search(query="What ML algorithms does he know?",k=3)

In [80]:
docs_ret[0]

Document(metadata={'source': 'C://Users//ASUS//Documents//Python Scripts//LnE Langchain//RAG Documents//Ankon Bhowmick Resume.docx'}, page_content='Certifications\n\nData Science Project-Based Learning and Internship Program Using R, Python Programming, And Tableau, StepUp Analytics, Online, (December 2020 - February 2021)\n\nMicrosoft Excel (Office 2019): Exam Reference #45320081\n\nSkills\n\nProgramming Languages: Python, R, SQL\n\nPython Libraries: Pandas, NumPy, Matplotlib, Seaborn, Scikit-learn, Transformers, OpenCV, PyTorch, TensorFlow\n\nR Libraries: dplyr, ggplot2, caret, glm, rpart, timeSeries, rstanarm\n\nML Algorithms: Linear Regression, Logistic Regression, KNN, Decision Trees, Naïve Bayes, K-Means, DBSCAN, Random Forest, PCA, Multi-level Regression\n\nDeep Learning: CNN, RNN, LSTM\n\nNLP: Text Preprocessing, Embeddings, Vector Database (FAISS), BERT (HuggingFace), GenAI (LangChain)\n\nOther Relevant Coursework: Time Series, Business Analytics, Data Structures & Algorithms,

In [83]:
len(docs_ret[2].page_content)

825

## Custom RAG Chain

In [12]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_rag
    | llm
    | StrOutputParser()
)

In [13]:
my_query='What kind of work has he done in Python?'

In [107]:
answer=rag_chain.invoke(my_query)

In [81]:
Markdown(answer)

The candidate possesses a robust understanding of various machine learning algorithms, including Linear Regression, Logistic Regression, KNN, Decision Trees, Naïve Bayes, K-Means, DBSCAN, Random Forest, PCA, and Multi-level Regression.

Their practical application of these algorithms is evident in several projects:

*   **Predictive Modelling:** In a project focused on predicting the success of an ICO, the candidate adeptly utilized **Random Forest Classifier (RFC)** and **Support Vector Classifier (SVC)** models, achieving competitive accuracy scores. They also optimized these models by fine-tuning hyperparameters and rigorously evaluating performance using metrics like precision, recall, F1-score, and ROC-AUC analysis.
*   **Deep Learning for Diverse Data Types:** The candidate has a strong command over handling diverse data types, demonstrated through their work with **CNN (Convolutional Neural Networks)** for image classification and **RNN (Recurrent Neural Networks)** for image captioning. They constructed and trained CNN models on datasets like TinyImageNet30 and implemented transfer learning. For image captioning, they built an encoder-decoder architecture using RNNs.
*   **Natural Language Processing (NLP):** Their NLP project involved extensive work with unstructured data, where they implemented and fine-tuned the **LLM BERT** for analyzing news articles.
*   **Optimization:** The candidate also applied **linear programming** to optimize the allocation of delivery robots, enhancing operational efficiency.
*   **General Predictive Models:** They have leveraged machine learning frameworks such as TensorFlow, PyTorch, and scikit-learn to build predictive models and optimize their performance across various academic and personal endeavors.

## Multi-Query

In [14]:
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_genq = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser

generate_queries = (
    prompt_genq
    | llm 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [ ]:
query_gen=generate_queries.invoke({'question':'What kind of work has he done in Python?'})

In [43]:
query_gen[0]

'What Python projects has he completed?'

## Parallel RAG

In [17]:
parallel_rag=(
    RunnableEach(bound=rag_chain)
    | (lambda x: "".join(x))
)

In [96]:
answers=parallel_rag.invoke(query_gen)

In [98]:
Markdown(answers)

The candidate has completed several projects utilizing Python and its libraries:

*   **Estimating Public Opinion on Gun Legislation using MRP:** This project involved collecting survey and population data, and conducting data cleaning, transformation, and data matching using Python packages like Pandas and NumPy. Results were presented using Matplotlib.
*   **Image classification using CNN and Image Captioning using RNN:** For this project, the candidate constructed basic NN and CNN classification models using TensorFlow and PyTorch. An encoder-decoder architecture-based model was built to generate captions for images.
*   **Using Markov Chain Monte Carlo to Predict Average Resting Heart Rate:** While the primary algorithm (MCMC) is conceptual, the context mentions data collection and analysis, and given the candidate's Python skills, it's highly probable Python was used for data handling.
*   **Building a Quantitative Momentum Investing Model:** This project established a data pipeline using Pandas and NumPy for cleaning, transforming, and integrating data.
*   **Predicting the Course of the Russo-Ukrainian War:** This involved collecting data through web scraping using Selenium and Beautiful Soup packages in Python. Data was stored as Tensor datasets and PyTorch dataloaders. The project leveraged Hugging Face’s Transformers library for fine-tuning BERT.The candidate possesses extensive experience working with Python, leveraging it across various projects and for diverse data-related tasks.

Their technical proficiency in Python is robust, encompassing a wide array of data manipulation and machine learning libraries such as Pandas, NumPy, Scikit-learn, and TensorFlow. They have also utilized specialized Python libraries like Matplotlib and Seaborn for data visualization, and Transformers, OpenCV, and PyTorch for advanced applications.

During an internship at PwC, the candidate applied Python and machine learning techniques to manage the entire lifecycle of a data science project, specifically identifying patterns and developing a decision tree model from a large dataset of incident reports. This involved conducting in-depth exploratory data analysis (EDA) and cleaning datasets using Python.

Furthermore, their work with unstructured data heavily relied on Python. An NLP project involved analyzing news articles, requiring text preprocessing, implementation of the LLM BERT, and fine-tuning the model using Python. Similarly, their expertise extends to image classification using CNN and RNN for image captioning, all developed with Python, demonstrating a strong command over handling diverse data types. They have also applied machine learning classifiers in academic projects, such as predicting ICO success, utilizing Python frameworks like TensorFlow, PyTorch, and Scikit-learn to build predictive models and optimize performance.

The candidate's skill set also includes advanced NLP concepts in Python, such as Text Preprocessing, Embeddings, Vector Databases (FAISS), BERT (HuggingFace), and Generative AI (LangChain). This comprehensive background underscores their deep engagement with Python for complex analytical and machine learning endeavors.The candidate has a robust background in Python, demonstrating its application across various programming and data-centric tasks.

Their contributions and achievements related to Python include:

*   **Core Program Implementation:** The candidate has implemented main programs using Python3, including a specific project where Python v3.8 was utilized in an integrated development environment like Spyder. This showcases their ability to design and execute programs in Python.
*   **Object-Oriented Programming Proficiency:** They have leveraged Python's object-oriented capabilities, indicating an understanding of creating and utilizing classes and objects for structured code development.
*   **Extensive Library Application:** The candidate has proficiently used a wide array of Python libraries to address diverse challenges. This includes:
    *   **Data Manipulation and Analysis:** Pandas and NumPy for handling raw data, performing elementary analysis, array manipulation, and computing statistical measures like mean and standard deviation.
    *   **Mathematical Operations:** The built-in `math` module for functions such as `pow`, `sqrt`, and `pi`.
    *   **Randomization:** The `random` module for generating random numbers.
    *   **Data Visualization:** Matplotlib and Seaborn for creating visualizations and presenting complex findings.
    *   **Machine Learning:** Scikit-learn for implementing various ML algorithms including Linear Regression, Logistic Regression, KNN, Decision Trees, Naïve Bayes, K-Means, DBSCAN, Random Forest, PCA, and Multi-level Regression.
    *   **Deep Learning:** PyTorch, TensorFlow, and OpenCV for developing CNN, RNN, and LSTM models.
    *   **Natural Language Processing (NLP):** Transformers, BERT (HuggingFace), and GenAI (LangChain) for tasks such as text preprocessing, embeddings, and working with vector databases (FAISS).
*   **Algorithm Development:** They have implemented complex algorithms, such as the Metropolis-Hastings Algorithm, using Python.
*   **Data Science Engagement:** The candidate participated in a "Data Science Project-Based Learning and Internship Program" specifically utilizing Python programming, demonstrating practical experience in data science workflows.
*   **Code Quality and Readability:** They adhere to Python's design philosophy, emphasizing code readability through significant indentation, which aids in maintaining clear and logical code for both small and large-scale projects.
*   **Data-Driven Problem Solving:** Their experience has taught them to approach problem-solving with a data-driven mindset, effectively using Python for analysis and visualization to derive insights.The candidate has demonstrated extensive use of Python across a multitude of areas and applications:

*   **Data Manipulation and Analysis:** Python, along with libraries like Pandas and NumPy, has been leveraged for managing and analyzing large datasets, including tasks such as data cleaning, transformation, and data matching.
*   **Machine Learning and Deep Learning:** The candidate is proficient in employing Python libraries such as Scikit-learn, TensorFlow, and PyTorch for implementing a wide array of machine learning algorithms (e.g., Linear Regression, Logistic Regression, KNN, Decision Trees, Naïve Bayes, K-Means, DBSCAN, Random Forest, PCA, Multi-level Regression) and deep learning models (CNN, RNN, LSTM).
*   **Natural Language Processing (NLP):** Python has been instrumental in NLP applications, encompassing text preprocessing, embeddings, utilizing vector databases (FAISS), and working with advanced models like BERT (HuggingFace) and GenAI (LangChain).
*   **Data Visualization and Presentation:** The candidate has utilized Python libraries such as Matplotlib and Seaborn to effectively visualize data and present complex findings.
*   **General Programming and Project Implementation:** Python 3 has been used for implementing core programs, including the application of object-oriented programming principles and the use of built-in modules like `math` and `random`.
*   **Specific Projects and Coursework:** Python was a cornerstone in a Data Science Project-Based Learning and Internship Program, and was specifically applied in a project to estimate public opinion on gun legislation using MRP. Its application is also implied across relevant coursework in areas such as Time Series, Business Analytics, Data Structures & Algorithms, and Statistical Learning.The candidate's professional work involving Python includes:

*   **Core Proficiency:** They possess technical proficiency in Python, alongside a wide range of data manipulation and machine learning libraries such as Pandas, NumPy, Scikit-learn, and TensorFlow.
*   **Data Handling & Visualization:** They have utilized Python packages like Pandas and NumPy for data cleaning, transformation, and data matching. Data visualization has been performed using Matplotlib in Python to present complex findings.
*   **Machine Learning & Deep Learning:** Their skills extend to various Python-based machine learning algorithms and deep learning frameworks, including CNN, RNN, and LSTM, as well as libraries like PyTorch and TensorFlow.
*   **Natural Language Processing (NLP):** They have experience with NLP tasks using Python, encompassing text preprocessing, embeddings, vector databases (FAISS), BERT (HuggingFace), and Generative AI (LangChain).
*   **Project-Based Application:** Their experience includes a Data Science Project-Based Learning and Internship Program where Python programming was a key component.
*   **Specific Project Example:** In a project estimating public opinion on gun legislation, Python packages Pandas and NumPy were used to collect, clean, transform, and match survey and population data, creating input datasets and poststratification frames. Results were then presented using Matplotlib in Python.

## Complete Chain

In [18]:
prompt_final_human=HumanMessagePromptTemplate.from_template('''
You are given information about the candidate that is relevant to the question. Here's the information : {context}
Answer this question about the candidate : {question}''')
prompt_final=ChatPromptTemplate.from_messages([system_prompt, prompt_final_human])

In [19]:
'''final_answer=(
    prompt_final
    | llm
    | StrOutputParser()
)'''

In [20]:
final_answer=(
    {'context':(generate_queries
               | parallel_rag),
     'question':RunnablePassthrough()
    }
    | prompt_final
    | llm
    | StrOutputParser()
)

In [117]:
test=final_answer_t.invoke(my_query)

In [108]:
finalanswer=final_answer.invoke({'context':answers,'question':my_query})

In [118]:
Markdown(test)

The candidate has engaged in a wide range of work using Python, demonstrating comprehensive experience and expertise. Their work includes:

*   **Core Program Implementation:** Implemented the main program for projects like the NAND Trees project using Python3, utilizing its object-oriented capabilities and built-in modules.
*   **Data Science & Analysis:**
    *   Conducted in-depth analysis of large datasets (e.g., over 50,000 SAP incident reports at PwC) using advanced data analysis techniques.
    *   Performed extensive data cleaning, transformation, and data matching using Python packages like Pandas and NumPy for projects such as estimating public opinion on gun legislation.
    *   Engaged in exploratory data analysis (EDA) during their PwC internship.
*   **Machine Learning & Deep Learning:**
    *   Developed and implemented various machine learning models, including a decision tree model for predicting incident resolution likelihood (achieving 97.7% accuracy).
    *   Applied deep learning models such as Convolutional Neural Networks (CNN) for image classification and Recurrent Neural Networks (RNN) for image captioning.
    *   Utilized and fine-tuned large language models (LLMs) like BERT for text classification and natural language processing tasks, often leveraging Hugging Face's Transformers library.
    *   Implemented complex models using Python-based frameworks like TensorFlow, PyTorch, and scikit-learn in research-intensive projects.
    *   Has experience with a variety of ML algorithms including Linear Regression, Logistic Regression, KNN, Decision Trees, and Random Forest.
*   **Natural Language Processing (NLP):**
    *   Collected data through web scraping using Python packages like Selenium and Beautiful Soup.
    *   Preprocessed text data for NLP projects, including analyzing news articles.
    *   Worked with Text Preprocessing, Embeddings, Vector Databases (FAISS), BERT (HuggingFace), and Generative AI (GenAI) using LangChain.
*   **Data Visualization:** Presented complex findings and project results using Matplotlib in Python.
*   **Object-Oriented Programming (OOP):** Applied OOP principles in their Python implementations, such as for the NAND Trees project.
*   **Project Management & Lifecycle:** Managed the entire lifecycle of a data science project during their internship at PwC, which involved applying Python and machine learning techniques from data acquisition to result communication.

They are proficient with a wide array of Python libraries essential for these tasks, including Pandas, NumPy, Scikit-learn, TensorFlow, Matplotlib, Seaborn, Transformers, OpenCV, PyTorch, Selenium, Beautiful Soup, and LangChain.

In [109]:
Markdown(finalanswer)

The candidate has performed a wide range of work in Python, demonstrating extensive proficiency across various domains:

1.  **Core Programming and Software Development:**
    *   Implemented main programs using Python 3, including specific projects utilizing Python v3.8 in IDEs like Spyder.
    *   Leveraged Python's object-oriented programming (OOP) capabilities for structured code development.
    *   Adhered to Python's design philosophy, emphasizing code readability through significant indentation.
    *   Utilized built-in modules like `math` (for functions like `pow`, `sqrt`, `pi`) and `random` (for generating random numbers).

2.  **Data Manipulation, Analysis, and Engineering:**
    *   **Data Cleaning, Transformation, and Matching:** Proficiently used Pandas and NumPy for managing large datasets, including tasks like data cleaning, transformation, data matching, and creating input datasets/poststratification frames (e.g., in the Public Opinion project).
    *   **Exploratory Data Analysis (EDA):** Conducted in-depth EDA and cleaned datasets, notably during an internship at PwC for incident reports.
    *   **Data Pipelines:** Established data pipelines for cleaning, transforming, and integrating data (e.g., in the Quantitative Momentum Investing Model).
    *   **Statistical Analysis:** Performed elementary analysis, array manipulation, and computed statistical measures like mean and standard deviation using Pandas and NumPy.
    *   **Web Scraping:** Collected data through web scraping using Selenium and Beautiful Soup packages (e.g., for the Russo-Ukrainian War prediction project).

3.  **Machine Learning and Deep Learning:**
    *   **Algorithm Implementation:** Implemented a wide array of machine learning algorithms using Scikit-learn, including Linear Regression, Logistic Regression, KNN, Decision Trees, Naïve Bayes, K-Means, DBSCAN, Random Forest, PCA, and Multi-level Regression.
    *   **Predictive Modeling:** Built predictive models and optimized performance (e.g., predicting ICO success).
    *   **Deep Learning Model Construction:** Constructed basic Neural Networks (NN), Convolutional Neural Networks (CNN), Recurrent Neural Networks (RNN), and Long Short-Term Memory (LSTM) models using TensorFlow, PyTorch, and OpenCV (e.g., for image classification and image captioning using an encoder-decoder architecture).
    *   **Algorithm Development:** Implemented complex algorithms like the Metropolis-Hastings Algorithm.

4.  **Natural Language Processing (NLP):**
    *   **Text Preprocessing:** Performed text preprocessing for analyzing unstructured data like news articles.
    *   **Language Models:** Implemented and fine-tuned large language models (LLMs) like BERT using Hugging Face's Transformers library.
    *   **Embeddings and Vector Databases:** Worked with advanced NLP concepts such as embeddings and vector databases (FAISS).
    *   **Generative AI:** Utilized Generative AI concepts and libraries like LangChain.
    *   **Specific Applications:** Applied NLP for analyzing news articles and for image captioning.

5.  **Data Visualization and Presentation:**
    *   Utilized Matplotlib and Seaborn to create visualizations, present complex findings, and display results (e.g., for the Public Opinion project).

6.  **Project Management and Lifecycle:**
    *   Managed the entire lifecycle of data science projects, from data collection and cleaning to model development and deployment.
    *   Participated in a "Data Science Project-Based Learning and Internship Program" specifically utilizing Python.

7.  **Specific Project Applications:**
    *   **Estimating Public Opinion on Gun Legislation:** Used Pandas, NumPy, and Matplotlib for data collection, cleaning, transformation, matching, and presentation.
    *   **Image Classification and Captioning:** Built CNN and RNN models with TensorFlow and PyTorch.
    *   **Quantitative Momentum Investing Model:** Established a data pipeline with Pandas and NumPy.
    *   **Predicting the Course of the Russo-Ukrainian War:** Involved web scraping with Selenium and Beautiful Soup, data storage as Tensor datasets and PyTorch dataloaders, and fine-tuning BERT with Hugging Face.
    *   **PwC Internship:** Applied Python and machine learning to identify patterns and develop a decision tree model from incident reports, involving extensive EDA and data cleaning.

In summary, the candidate has leveraged Python as a primary tool for data science, machine learning, deep learning, NLP, data engineering, and general programming, applying it across diverse academic and professional projects.

In [110]:
Markdown(answer)

The candidate has performed a variety of work in Python, demonstrating a robust technical proficiency.

Their experience includes:
*   **Data Manipulation and Analysis:** They have utilized Python packages such as Pandas and NumPy for data cleaning, transformation, and data matching, as evidenced in a project estimating public opinion on gun legislation.
*   **Machine Learning and Deep Learning:** The candidate is proficient with Python libraries like Scikit-learn and TensorFlow. Their skills extend to deep learning algorithms including CNN, RNN, and LSTM.
*   **Natural Language Processing (NLP):** They have worked with Python-based NLP techniques, including text preprocessing, embeddings, vector databases (FAISS), BERT (HuggingFace), and Generative AI (LangChain).
*   **Data Visualization:** The candidate has used Matplotlib in Python to present complex findings, specifically for visualizing results in the public opinion estimation project.
*   **General Programming and Project Implementation:** They have implemented main programs in Python3, leveraging its object-oriented capabilities and built-in modules like `math` and `random`.
*   **Certifications and Programs:** Their skills were further honed through a Data Science Project-Based Learning and Internship Program that extensively used Python programming.

The candidate's listed Python libraries include Pandas, NumPy, Matplotlib, Seaborn, Scikit-learn, Transformers, OpenCV, PyTorch, and TensorFlow, indicating a broad practical application of the language.

## Async Parallel

In [23]:
answer_async= await final_answer.ainvoke(my_query)

In [24]:
answer=final_answer.invoke(my_query)